### Бриф
От клиента поступил заказ на исследование восприятия бренда. Универсальные решения плохо учитывают специфику продукции, поэтому бренд хочет провести свое собственное исследование.

С помощью крауд сорсинга мы разметили датасет из нескольких тысяч отзывов, на этих размеченых отзывах обучили классификатор тональности (отзыв => позитивный/негативный) и оценили его точность. 

Заказчик хочет знать, какие харакетристики продукта или слова являются ключевыми для хороших отзывов. Чтобы ответить на этот вопрос, необходимо сконвертировать текст отзыва в формат вектора фичей, рассчитать их значимость любым доступным способом и вывести топ наиболее важных слов.

### Данные
см HW_NLP_1

### Требования
1. Результат должен быть предоставлен в виде ноутбука с python кодом

### Оценка
- выполнены базовые требования => 7 баллов
- доп балл, если используется датасет, который не повторяется у других студентов (см. HW_NLP_1)
- доп балл, если значимость рассчитана более, чем одним показателем
- доп балл, если все чисто и красиво оформлено с комментариями


In [12]:
import pandas as pd

#Загружаем данные
df = pd.read_csv('/Users/denis/Downloads/amazon_alexa.tsv', sep='\t')
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [13]:

df = df[['verified_reviews','rating']]
df = df.dropna()

#Деление на тест-сплит
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['verified_reviews'], df['rating'], test_size=0.33, random_state=42)


In [14]:
#Векторизуем слова
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# Считаем вхождения термов в каждый документ
X_train_counts = count_vect.fit_transform(X_train)

# Взвешиваем полученное векторное представление с помощью TF-IDF преобразования
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [15]:
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
# Обучение классификатора
RFclf = RandomForestClassifier(random_state=0, max_depth=20, 
                               max_features='sqrt', min_samples_leaf=4, min_samples_split=10, 
                               n_estimators=200)

rfe = RFE(estimator=RFclf, n_features_to_select = 10, step=1000).fit(X_train_tfidf, y_train)

In [16]:
words_rfe = pd.DataFrame(count_vect.vocabulary_.keys(),columns=['word'])
words_rfe['rank'] = rfe.ranking_
words_rfe['importance'] = rfe.support_
words_rfe[words_rfe['rank'] == 1]


,word,rank,importance
223,little,1,True
362,loved,1,True
491,programs,1,True
758,how,1,True
965,moment,1,True
1681,stereo,1,True
1690,cover,1,True
1884,save,1,True
2099,components,1,True
3492,guarantee,1,True


10 самых важных слов

In [17]:
from sklearn.ensemble import RandomForestClassifier
RFclf = RandomForestClassifier(random_state=0, max_depth=28, 
                               max_features='sqrt', min_samples_leaf=4, min_samples_split=10, 
                               n_estimators=692).fit(X_train_tfidf, y_train)

words_rf_fi = pd.DataFrame(count_vect.vocabulary_.keys(),columns=['word'])
words_rf_fi['importance'] = RFclf.feature_importances_
words_rf_fi.sort_values('importance',ascending = False).head(20)

,word,importance
1884,save,0.040037
2099,components,0.037955
491,programs,0.019565
965,moment,0.014634
3492,guarantee,0.014492
362,loved,0.013996
758,how,0.013464
1681,stereo,0.012212
3506,hundreds,0.011008
398,didnt,0.010918


Важные фичи Random Forest Classifier